# Deploying the Model of the Best Automatic Model Tuning Training Job

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
from sagemaker import tuner
import sagemaker

session = sagemaker.Session()

In [ ]:
%store -r tuning_job_name

In [ ]:
%store -r s3_bucket_name

In [ ]:
hyperparameter_tuner = tuner.HyperparameterTuner.attach(
    tuning_job_name, 
    sagemaker_session=session)

In [ ]:
%%time

predictor = hyperparameter_tuner.deploy(
    initial_instance_count=1, 
    instance_type='ml.m5.large',
    endpoint_name='tuner-best-model-endpoint'
)

In [ ]:
!mkdir -p tmp

In [ ]:
!aws s3 cp s3://{s3_bucket_name}/chapter06/input/test_data_no_header.csv tmp/test_data_no_header.csv

In [ ]:
import pandas as pd

test_data = pd.read_csv("tmp/test_data_no_header.csv", header=None)
test_data

In [ ]:
test_data_no_target = test_data.drop(test_data.columns[0], axis=1)
test_data_no_target

In [ ]:
csv_input = test_data_no_target.iloc[0:1].to_csv(index=False, header=False)

In [ ]:
csv_input

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()

predictor.predict(csv_input)

In [ ]:
predictor.delete_endpoint()